In [26]:
import pandas as pd
dat = pd.read_csv(r"C:\Users\camda\OneDrive\Desktop\Kaggle\titanic\train.csv")
dat2 = pd.read_csv(r'C:\Users\camda\OneDrive\Desktop\Kaggle\titanic\test.csv')
pd.set_option('display.max_rows', False)

In [27]:
dat1 = pd.DataFrame(dat)
split = len(dat1)
Y = dat1['Survived']
dat1 = dat1.drop(['Survived'],axis=1)
df = pd.concat([dat1,pd.DataFrame(dat2)])

df['Gender'] = (df['Sex']=='male').astype(int)
df = df.drop(['Name','Parch','Sex','Ticket','Embarked','Cabin','Age'],axis=1).fillna(0)



In [28]:
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
import numpy as np
x = df.drop(['PassengerId'], axis=1)
x_train = x[:split]
y_train = Y
x_test = x[split:]



In [29]:
scaler = preprocessing.StandardScaler().fit(x_train)
x_scaled = scaler.transform(x_train)
x_scaled_test = scaler.transform(x_test)

In [30]:
model = LogisticRegression()
model.fit(x_scaled,y_train)
y_pred = model.predict_proba(x_scaled_test)
low = 0.3
upper = 0.7
final_predictions = np.full(len(x_test), np.nan)
uncertain_indices = []
for i, prob in enumerate(y_pred[:, 1]):
    if prob <= low:
        final_predictions[i] = 0
    elif prob >= upper:
        final_predictions[i] = 1
    else:
        uncertain_indices.append(i)

In [31]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state=0,criterion='gini')
clf.fit(x_train,y_train)
uncertain_x = x_test.iloc[uncertain_indices]
y_pred2 = clf.predict(uncertain_x)
for i, index in enumerate(uncertain_indices):
    final_predictions[index] = y_pred2[i]

In [34]:
result = pd.DataFrame([df['PassengerId'][split:]])
result = pd.DataFrame(df['PassengerId'][split:])
result['Survived'] = final_predictions.astype(int)
result.to_csv('titanicresults.csv',index=False)
